In [1]:
# importing required libraries:
import numpy as np
import cv2
import os
import random
import matplotlib.pyplot as plt
import pickle

In [2]:
# setting the path to our eye dataset: 
Directory = "/Users/anikashrivastava/Downloads/driver-drowsiness-detection-Project/dataset_new/train"
# specify two categories on which we want to train our data:
CATEGORIES = ['Closed' , 'Open']

In [3]:
#setting image size:
img_size = 24
data = []

#iterating over each image and get the image in array form,
for category in CATEGORIES:
    folder = os.path.join(Directory,category)
    label = CATEGORIES.index(category)
    for img in os.listdir(folder):
        img_path = os.path.join(folder, img)
        img_arr = cv2.imread(img_path)
        img_arr = cv2.cvtColor(img_arr, cv2.COLOR_BGR2GRAY)
        img_arr = cv2.resize(img_arr,(img_size, img_size),1)
        data.append([img_arr , label])

In [14]:
np.shape(data[0][0])

(24, 24)

In [4]:
# see the length of data:
len(data)

1234

In [5]:
# we shuffle the data to get random images of open eyes and closed eyes:
random.shuffle(data)

In [7]:
# dividing features and label for training the model: 
X = []
Y = []

for features,label in data:
    X.append(features)
    Y.append(label)

In [8]:
#covert them into array:
X = np.array(X)
Y = np.array(Y)

In [15]:
# save the data into system:
pickle.dump(X , open('X.pkl' , 'wb'))
pickle.dump(Y , open('Y.pkl' , 'wb'))

In [16]:
# normalize the image array:
X = X/255

In [17]:
X

array([[[0.85098039, 0.82352941, 0.78431373, ..., 0.76470588,
         0.7372549 , 0.61176471],
        [0.83921569, 0.79215686, 0.76862745, ..., 0.8       ,
         0.75294118, 0.68235294],
        [0.82745098, 0.79607843, 0.75686275, ..., 0.79215686,
         0.76470588, 0.72941176],
        ...,
        [0.85098039, 0.85882353, 0.83529412, ..., 0.81568627,
         0.81568627, 0.81176471],
        [0.85098039, 0.85098039, 0.83137255, ..., 0.81176471,
         0.83137255, 0.82745098],
        [0.85098039, 0.84313725, 0.84313725, ..., 0.82745098,
         0.83137255, 0.83137255]],

       [[0.82745098, 0.81176471, 0.81568627, ..., 0.64705882,
         0.7254902 , 0.71764706],
        [0.80392157, 0.79215686, 0.79215686, ..., 0.65882353,
         0.70980392, 0.70196078],
        [0.78823529, 0.76078431, 0.7372549 , ..., 0.63529412,
         0.69411765, 0.71764706],
        ...,
        [0.76078431, 0.73333333, 0.70196078, ..., 0.64705882,
         0.7254902 , 0.74509804],
        [0.7

In [18]:
# reshape the X array to (24,24,1)
img_rows,img_cols = 24,24
X = X.reshape(X.shape[0],img_rows,img_cols,1)
X.shape

(1234, 24, 24, 1)

In [19]:
# we will be using keras to create the model:
from keras.models import Sequential
from keras.layers import Conv2D , MaxPooling2D , Flatten , Dense

In [20]:
# creating model:
model = Sequential()

model.add(Conv2D(64 , (3,3) , activation = 'relu' , input_shape= X.shape[1:]))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D((1,1)))

model.add(Flatten())

model.add(Dense(128, activation = 'relu'))

model.add(Dense(2, activation = 'softmax'))

In [21]:
# compile model that we have created
model.compile(optimizer = 'adam' , loss = 'sparse_categorical_crossentropy' , metrics = ['accuracy'])

In [22]:
# fit X , Y to the model to see accuracy of model:
model.fit(X, Y, epochs = 5 , validation_split = 0.1 , batch_size = 32)

Epoch 1/5


2025-01-22 15:30:55.976599: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


35/35 [==============================] - 1s 28ms/step - loss: 0.4065 - accuracy: 0.8108 - val_loss: 0.1903 - val_accuracy: 0.9194
Epoch 2/5
35/35 [==============================] - 1s 25ms/step - loss: 0.1467 - accuracy: 0.9441 - val_loss: 0.2149 - val_accuracy: 0.9113
Epoch 3/5
35/35 [==============================] - 1s 25ms/step - loss: 0.1081 - accuracy: 0.9631 - val_loss: 0.1342 - val_accuracy: 0.9435
Epoch 4/5
35/35 [==============================] - 1s 25ms/step - loss: 0.0823 - accuracy: 0.9694 - val_loss: 0.1551 - val_accuracy: 0.9435
Epoch 5/5
35/35 [==============================] - 1s 25ms/step - loss: 0.0728 - accuracy: 0.9766 - val_loss: 0.0899 - val_accuracy: 0.9677


In [23]:
# save model and architecture to single file
model.save("custmodel1.h5")